In [ ]:
import pandas as pd
import ast
import numpy as np
from statistics import mean

data_papers = pd.read_csv('papers.csv')
data_authors = pd.read_csv('authors.csv')
data_orgs = pd.read_csv('orgs.csv')

# 
data_papers['self_cite_percent'] = ""
data_papers['journal_percent'] = ""
data_papers['conference_percent'] = ""
data_papers['patent_percent'] = ""
data_papers['book_percent'] = ""
data_papers['other_percent'] = ""
data_papers['productivity'] = ""
data_papers['title_length'] = ""
data_papers['H_index'] = ""
data_papers['max_H_index'] = ""
data_papers['references_ave_age'] = ""

# 
data_papers['journal_publications'] = ""
data_papers['organization_num'] = "" 
data_papers['prev_citation_sum'] = ""
data_papers['faculty_impact'] = "" 

# 
data_papers['paper_len'] = ""
data_papers['n_authors'] =  "" 
data_papers['avg_cite'] = ""
data_papers['n_papers'] = ""
data_papers['n_references'] = ""



for i in range(len(data_papers)):
    
    # 
    percentages = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
    
    if str(data_papers['references'][i]) != 'nan':
        references_paper = ast.literal_eval(data_papers['references'][i])
        
        for j in references_paper:
            type_doc = data_papers.loc[data_papers['id'] == np.int64(j)]['doc_type'].tolist()
            if 'Journal' in type_doc:
                percentages[1] += 1
            elif 'Conference' in type_doc:
                percentages[2] += 1
            elif 'Patent' in type_doc:
                percentages[3] += 1
            elif 'Book' in type_doc:
                percentages[4] += 1
            elif len(type_doc) != 0:
                if str(type_doc[0]) != 'nan':
                    print(type_doc)
                    percentages[5] += 1
            
            publish_doc = data_papers.loc[data_papers['id'] == np.int64(j)]['publisher'].tolist()
            if data_papers['publisher'][i] in publish_doc:
                percentages[0] += 1
        
        if len(references_paper) != 0:  
            percentages = [x * 1.0 / len(references_paper) for x in percentages]
       
        data_papers['self_cite_percent'][i] = percentages[0]
        data_papers['journal_percent'][i] = percentages[1]
        data_papers['conference_percent'][i] = percentages[2]
        data_papers['patent_percent'][i] = percentages[3]
        data_papers['book_percent'][i] = percentages[4]
        data_papers['other_percent'][i] = percentages[5]
    
    # 
    # journal_publications = the average number of papers published by an orgaization in one year
    authorsOrg = data_authors.loc[data_authors['id'] == data_papers['first_author'][i]]['org'].tolist()
    if str(authorsOrg[0]) != 'nan':
        data_papers['journal_publications'][i] = data_orgs.loc[data_orgs['name'] == authorsOrg[0]]['journal_publications'].tolist()[0]
    
    # organization_num = len(set(author.org))
    authors_ids = ast.literal_eval(data_papers['authors'][i])
    authors = []
    for ids in authors_ids:
        selected_author_org = data_authors.loc[data_authors['id'] == np.int64(ids), 'org'].tolist() 
        if (str(selected_author_org[0]) !=  'nan'):
            authors.extend(selected_author_org)
            authors = list(set(authors))
    data_papers['organization_num'][i] = len(authors)
    
    # prev_citation_sum = Previous citations
    author = ast.literal_eval(data_papers['authors'][i])
    papers = ast.literal_eval(data_authors.loc[data_authors['id'] == np.int64(author[0])]['papers'].tolist()[0])
    sum_cite = 0
    for x in papers:
        # print(str(x))
        paper = data_papers.loc[data_papers['id'] == np.int64(x)]
        print(paper)
        n_citations = paper.loc[paper['year'] <= np.int64(data_papers['year'][i])]['n_citation'].tolist()
        print("n_citations\t\t", n_citations)
        print("this\t", paper.loc[paper['year'] <= np.int64(data_papers['year'][i])])
        print("ttt\t", np.int64(data_papers['year'][i]))
        print(len(paper))
        if len(paper) != 0:
            if len(paper) != 1:
                print("\t", paper['year'][0])
                print("\t", type(paper['year'][0]))
            else:
                print("\t", paper['year'])
                print("\t", type(paper['year']))
        if len(n_citations) != 0:
            sum_cite += sum(n_citations)
    
    # faculty_impact = The average h-index among the authors affiliated with the institution of the first author
    author = ast.literal_eval(data_papers['authors'][i])
    org = data_authors.loc[data_authors['id'] == np.int64(author[0])]['org'].tolist()[0]
    if(str(org) != 'nan'):
        data_papers['faculty_impact'][i] = (data_orgs.loc[data_orgs['name'] == org]['ave_H_index'].tolist())[0]

    #
    authorsPapers = (data_authors.loc[data_authors['id'] == data_papers['first_author'][i]]['papers'].tolist())[0]
    data_papers['productivity'][i] = len(ast.literal_eval(authorsPapers)) 
    
    data_papers['title_length'][i] = len(data_papers['title'][i].split())
    
    data_papers['H_index'][i] = (data_authors.loc[data_authors['id'] == data_papers['first_author'][i]]['H_index'].tolist())[0]
    
    author = ast.literal_eval(data_papers['authors'][i])
    org = data_authors.loc[data_authors['id'] == np.int64(author[0])]['org'].tolist()[0]
    if(str(org) != 'nan'):
        data_papers['max_H_index'][i] = (data_orgs.loc[data_orgs['name'] == org]['max_H_index'].tolist())[0]
    
    if str(data_papers['references'][i]) != 'nan':
        sumYears = 0.0
        numYears = 0.0
        for ref in ast.literal_eval(data_papers['references'][i]):
            year = data_papers.loc[data_papers['id'] == np.int64(ref)]['year'].tolist()
            if len(year) != 0:
                sumYears += int(year[0])
                numYears += 1.0
        if numYears != 0:
            data_papers['references_ave_age'][i] = 2020 - sumYears/numYears
    
    
    # 
    # Length of Paper - Casting as int() results in an error
    if str(data_papers['page_end'][i]) != 'nan' and str(data_papers['page_start'][i]) != 'nan':
        data_papers['paper_len'][i] = (np.float64(str(data_papers['page_end'][i]).replace('-', '')) - np.float64(str(data_papers['page_start'][i]).replace('-', ''))+1)
    
    # Number of Authors 
    data_papers['n_authors'][i] = len(ast.literal_eval(data_papers['authors'][i]))
    
    # # Previous Citations per Article
    if data_papers['productivity'][i] != 0 and data_papers['productivity'][i] != "" and data_papers['prev_citation_sum'][i] != "":   
        data_papers['avg_cite'][i] = data_papers['prev_citation_sum'][i] / data_papers['productivity'][i]  

    # Total Number of Papers Produced in the Institute 
    author = ast.literal_eval(data_papers['authors'][i])
    org = data_authors.loc[data_authors['id'] == np.int64(author[0])]['org'].tolist()[0]
    if str(org) != 'nan':
        data_papers['n_papers'][i] = (data_orgs.loc[data_orgs['name'] == org]['num_of_papers'].tolist())[0]

    # Number of References
    if str(data_papers['references'][i]) != 'nan':  
        data_papers['n_references'][i] = len(ast.literal_eval(data_papers['references'][i]))

fileName = 'features.csv'
data_papers[['id', 'doc_type', 'journal_publications', 'organization_num', 'prev_citation_sum', 'faculty_impact', 'productivity', 'title_length', 'H_index', 'max_H_index', 'references_ave_age', 'self_cite_percent', 'journal_percent', 'conference_percent', 'patent_percent', 'book_percent', 'paper_len', 'avg_cite', 'n_papers', 'n_references', 'n_authors', 'n_citation']].to_csv(fileName, encoding='utf-8')